In [1]:
from model import *
import pytchat


C:\Users\avika\OneDrive\Documents\UAL\interactive_dance_thesis


In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="michellejieli/emotion_text_classifier")
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("michellejieli/emotion_text_classifier")
model = AutoModelForSequenceClassification.from_pretrained("michellejieli/emotion_text_classifier")

In [3]:
# Preparing MEED data for model
MEED= prep_MEED_data()
train_data,val_data,frame_dim,max_x,min_x,max_y,min_y,max_dx,min_dx,max_dy,min_dy = MEED

100%|██████████| 4102/4102 [00:03<00:00, 1054.20it/s]


Validating interpolation...
No errors found!


4102it [00:01, 2416.58it/s]
4102it [00:01, 2270.88it/s]


Creating keypoint frames...


100%|██████████| 4102/4102 [00:03<00:00, 1199.06it/s]


...

Validating length of dkp_frames
Validating length of kp_frames
Adding deltas to frames...


4102it [00:02, 1784.76it/s]


Validating length of data after delta
Adding emotions to frames...


100%|██████████| 4102/4102 [00:00<00:00, 34903.89it/s]


Validating length of data after emotions
frame_dim: 107


In [4]:
def get_video_by_emotion(data, specific_emotion):
    """
    Retrieve a single video of a specified emotion from the dataset.
    
    Parameters:
    - data (list): The entire dataset, assumed to be a list of videos.
    - specific_emotion (str): The emotion label of the desired video.
    
    Returns:
    - list: A single video corresponding to the specified emotion.
    """
    # Convert the specific emotion to its encoding
    specific_emotion_tuple = tuple(emotion_to_encoding(specific_emotion))
    
    # Search the dataset for a video with the specified emotion - return as tensor
    for video in data:
        if tuple(video[0][-7:]) == specific_emotion_tuple:
            return torch.tensor([video]).to(device).float()
    
    # If no video with the specified emotion is found, return None
    return None




# Example usage
specific_emotion_video = get_video_by_emotion(train_data, "Anger")

In [5]:
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

In [23]:
chat = pytchat.create(video_id="jfKfPfyJRdk")

# Initialize a dictionary to keep track of cumulative scores and counts for each emotion
emotion_data = {emotion: {"score": 0.0, "count": 0} for emotion in emotion_labels}

# List to store average scores indexed by emotions
average_scores = [0.0 for _ in emotion_labels]
length =0


# create model
CHECKPOINT_PATH = "checkpoints/proto5_checkpoint.pth"

m = MotionModel(input_dim=frame_dim, output_dim=frame_dim, blocksize=BLOCK_SIZE, hidden_dim=512, n_layers=8, dropout=DROPOUT)
m = m.to(device)
optimizer = torch.optim.Adam(m.parameters(), lr=LEARNING_RATE, weight_decay=L2_REG)  # weight_decay=1e-5 L2 regularization
m, optimizer,scheduler, epoch, loss, train_seed = load_checkpoint(m, optimizer, CHECKPOINT_PATH)

# initial generation always neutral
generated = m.generate(specific_emotion_video, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)

# visualize
visualise_skeleton(unnorm_out[0], max_x, max_y, max_frames=FRAMES_GENERATE,save = False,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust',train_seed=train_seed,delta=False)


# have to normalise the generated data before putting it into the model

Loading checkpoint...
Checkpoint loaded from checkpoints/proto5_checkpoint.pth


Visualizing frames: 100%|██████████| 300/300 [00:32<00:00,  9.18it/s]


In [40]:
def normalise_generated(unnorm_out, max_x, min_x, max_y, min_y, max_dx, min_dx, max_dy, min_dy):
    norm_out = []
    
    for frame in unnorm_out:
        norm_frame = []
        
        # Normalize the first 50 values (absolute x and y coordinates)
        for i in range(0, 50, 2):
            unnormalized_x = frame[i]
            unnormalized_y = frame[i+1]
            
            norm_x = 2 * (unnormalized_x - min_x) / (max_x - min_x) - 1
            norm_y = 2 * (unnormalized_y - min_y) / (max_y - min_y) - 1
            
            norm_frame.extend([norm_x, norm_y])
        
        # Normalize the second 50 values (x and y deltas)
        for i in range(50, 100, 2):
            unnormalized_dx = frame[i]
            unnormalized_dy = frame[i+1]
            
            norm_dx = 2 * (unnormalized_dx - min_dx) / (max_dx - min_dx) - 1
            norm_dy = 2 * (unnormalized_dy - min_dy) / (max_dy - min_dy) - 1
            
            norm_frame.extend([norm_dx, norm_dy])
        
        # Append the emotion encoding without normalizing
        norm_frame.extend(frame[-7:])
        norm_out.append(norm_frame)
        
    return norm_out


In [27]:
np.shape(unnorm_out[0][-16:])

(16, 107)

In [41]:
# testing normalise_generated

average_scores= [0.0, 0.0, 0.0, 0.0, 0.9258161187171936, 0.0, 0.0]

# Generate a new batch based on the current average scores
last_frames = unnorm_out[0][-16:]
norm_last_frames = normalise_generated(last_frames, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)
new_batch = [frame[:-7] + average_scores for frame in norm_last_frames]
new_input = torch.tensor([new_batch]).to(device).float()

# Generate the new frames

generated = m.generate(new_input, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)

# visualize
visualise_skeleton(unnorm_out[0], max_x, max_y, max_frames=FRAMES_GENERATE,save = False,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust',train_seed=train_seed,delta=False)


Visualizing frames: 100%|██████████| 300/300 [00:32<00:00,  9.13it/s]


In [46]:
%%timeit

generated = m.generate(new_input, FRAMES_GENERATE)

3.91 s ± 24.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
import threading
import queue
import time

In [48]:
chat = pytchat.create(video_id="6v3j-BGsYDg")
FRAMES_GENERATE = 100
shared_data = {
    'average_scores': [0] * len(emotion_labels)
}

def generate_new_batch():
    # prep new input from chat emotion average scores and last 16 frames of generated
    last_frames = unnorm_out[0][-16:]
    new_batch = [frame[:-7] + average_scores for frame in unnorm_out[0][-16:]]
    new_batch = torch.tensor(new_batch).to(device).float()

    # generate new frames
    generated = m.generate(new_batch, FRAMES_GENERATE)
    return unnormalise_list_2D(generated, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)

def visualize(unnorm_out):
    # visualize
    visualise_skeleton(unnorm_out[0], max_x, max_y, max_frames=FRAMES_GENERATE, save=False, save_path=None, prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust', train_seed=train_seed, delta=False)


# This queue will hold the batches ready for visualization
viz_queue = queue.Queue()

def visualize_batches():
    while True:
        # Wait for a new batch to be available in the queue
        unnorm_out = viz_queue.get()
        
        # Visualize the batch
        visualize(unnorm_out)
        


def generate_batches_periodically(period=10):
    while True:
        # Wait for the specified period
        time.sleep(period)
        
        # Generate a new batch based on the current average scores
        last_frames = unnorm_out[0][-16:]
        norm_last_frames = normalise_generated(last_frames, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)
        new_batch = [frame[:-7] + average_scores for frame in norm_last_frames]
        new_input = torch.tensor([new_batch]).to(device).float()

        # Generate the new frames
        generated = m.generate(new_input, FRAMES_GENERATE)
        unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)

        # Put the batch in the visualization queue
        viz_queue.put(unnorm_out)

# Start the periodic batch generation thread
threading.Thread(target=generate_batches_periodically, args=(10,), daemon=True).start()


# Start the visualization thread
threading.Thread(target=visualize_batches, daemon=True).start()


# issue is if chat isnt active this wont move much
while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")
        result = pipe(c.message)  # Assuming pipe() returns emotion prediction
        print(result)
        
        # Update emotion data with new score and count
        emotion = result[0]['label']
        score = result[0]['score']
        emotion_data[emotion]["score"] += score
        emotion_data[emotion]["count"] += 1
        
        # Compute average scores for each emotion
        for i, emotion in enumerate(emotion_labels):
            if emotion_data[emotion]["count"] > 0:
                average_scores[i] = emotion_data[emotion]["score"] / emotion_data[emotion]["count"]
        
        # Print average scores (can also store them elsewhere if needed)
        print("Average scores:", average_scores)
        length  +=1
        
        if length % 1 == 0:
            # Generate a new batch based on the current average scores
            last_frames = unnorm_out[0][-16:]
            norm_last_frames = normalise_generated(last_frames, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)
            new_batch = [frame[:-7] + average_scores for frame in norm_last_frames]
            new_input = torch.tensor([new_batch]).to(device).float()


            # Generate the new frames
            generated = m.generate(new_input, FRAMES_GENERATE)
            unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)

            # Put the batch in the visualization queue
            viz_queue.put(unnorm_out)



2023-10-19 21:42:23 [Moon Rabbit]- I really hate you you know that
[{'label': 'disgust', 'score': 0.49187198281288147}]
Average scores: [0.5959455711500985, 0.4970051467418671, 0.0, 0.8157621166922829, 0.8821436061213414, 0.8616082146763802, 0.6938560158014297]


Visualizing frames:  31%|███       | 31/100 [00:03<00:07,  9.25it/s]

2023-10-19 21:42:29 [Moon Rabbit]- you suck so much as a person
[{'label': 'joy', 'score': 0.9237198829650879}]
Average scores: [0.5959455711500985, 0.4970051467418671, 0.0, 0.8176899339471545, 0.8821436061213414, 0.8616082146763802, 0.6938560158014297]


Visualizing frames:  84%|████████▍ | 84/100 [00:09<00:01,  9.21it/s]

2023-10-19 21:42:34 [Moon Rabbit]- I hope you go to hell
[{'label': 'disgust', 'score': 0.5716186165809631}]
Average scores: [0.5959455711500985, 0.5094407250483831, 0.0, 0.8176899339471545, 0.8821436061213414, 0.8616082146763802, 0.6938560158014297]


...

Visualizing frames:  39%|███▉      | 39/100 [00:04<00:06,  9.23it/s]

2023-10-19 21:47:21 [Moon Rabbit]- happy
[{'label': 'joy', 'score': 0.9400689005851746}]
Average scores: [0.5959455711500985, 0.5150168687105179, 0.0, 0.834107839747479, 0.8815746638646795, 0.8450762900439176, 0.6938560158014297]


Visualizing frames:  26%|██▌       | 26/100 [00:02<00:07,  9.31it/s]

2023-10-19 21:47:53 [Moon Rabbit]- happy happy happy happy happy happy happy happy happy
[{'label': 'joy', 'score': 0.9782267212867737}]
Average scores: [0.5959455711500985, 0.5150168687105179, 0.0, 0.8359795135337037, 0.8815746638646795, 0.8450762900439176, 0.6938560158014297]


Visualizing frames: 100%|██████████| 100/100 [00:10<00:00,  9.23it/s]


In [49]:
import time
import threading

# Initial setup
shared_data = {
    'average_scores': [0] * len(emotion_labels)
}

chat = pytchat.create(video_id="6v3j-BGsYDg")
FRAMES_GENERATE = 100

# Emotion processing function
def process_chat_message(c):
    """Process a chat message and update emotion scores."""
    print(f"{c.datetime} [{c.author.name}]- {c.message}")
    result = pipe(c.message)  # Assuming pipe() returns emotion prediction
    print(result)

    emotion = result[0]['label']
    score = result[0]['score']
    emotion_data[emotion]["score"] += score
    emotion_data[emotion]["count"] += 1

    # Update average scores
    for i, emotion in enumerate(emotion_labels):
        if emotion_data[emotion]["count"] > 0:
            shared_data['average_scores'][i] = emotion_data[emotion]["score"] / emotion_data[emotion]["count"]

    print("Average scores:", shared_data['average_scores'])

# Batch generation function
def generate_new_batch():
    """Generate a new batch based on the current average scores."""
    last_frames = unnorm_out[0][-16:]
    norm_last_frames = normalise_generated(last_frames, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)
    new_batch = [frame[:-7] + shared_data['average_scores'] for frame in norm_last_frames]
    new_input = torch.tensor([new_batch]).to(device).float()

    # Generate the new frames
    generated = m.generate(new_input, FRAMES_GENERATE)
    return unnormalise_list_2D(generated, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)

# Periodic batch generation function
def generate_batches_periodically(period=10):
    """Periodically generate batches and put them in the visualization queue."""
    while True:
        time.sleep(period)
        unnorm_out = generate_new_batch()
        viz_queue.put(unnorm_out)

# Visualization thread
def visualize_batches():
    while True:
        # Wait for a new batch to be available in the queue
        unnorm_out = viz_queue.get()
        
        # Visualize the batch
        visualize(unnorm_out)

# Start the threads
threading.Thread(target=visualize_batches, daemon=True).start()
threading.Thread(target=generate_batches_periodically, args=(10,), daemon=True).start()

# Process chat messages
length = 0
while chat.is_alive():
    for c in chat.get().sync_items():
        process_chat_message(c)
        length += 1


Visualizing frames: 100%|██████████| 100/100 [00:10<00:00,  9.23it/s]


Visualizing frames:   0%|          | 0/100 [00:00<?, ?it/s]

In [7]:

while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")
        result = pipe(c.message)  # Assuming pipe() returns emotion prediction
        print(result)
        
        # Update emotion data with new score and count
        emotion = result[0]['label']
        score = result[0]['score']
        emotion_data[emotion]["score"] += score
        emotion_data[emotion]["count"] += 1
        
        # Compute average scores for each emotion
        for i, emotion in enumerate(emotion_labels):
            if emotion_data[emotion]["count"] > 0:
                average_scores[i] = emotion_data[emotion]["score"] / emotion_data[emotion]["count"]
        
        # Print average scores (can also store them elsewhere if needed)
        print("Average scores:", average_scores)
        length  +=1
        
        if length % 10 ==0 :
            generated = m.generate(xb, FRAMES_GENERATE)


2023-10-19 18:55:18 [ Ar. Sharma~]- :cigarette:
[{'label': 'neutral', 'score': 0.9502205848693848}]
Average scores: [0.0, 0.0, 0.0, 0.0, 0.9502205848693848, 0.0, 0.0]
2023-10-19 18:55:18 [Michael Terry Wilson]- Best wishes and love to everyone.
[{'label': 'neutral', 'score': 0.682776689529419}]
Average scores: [0.0, 0.0, 0.0, 0.0, 0.8164986371994019, 0.0, 0.0]


KeyboardInterrupt: 

In [ ]:
# Preparing MEED data for model
MEED= prep_MEED_data()
train_data,val_data,frame_dim,max_x,min_x,max_y,min_y,max_dx,min_dx,max_dy,min_dy = MEED

In [2]:


CHECKPOINT_PATH = "checkpoints/proto5_checkpoint.pth"

# create model
m = MotionModel(input_dim=frame_dim, output_dim=frame_dim, blocksize=BLOCK_SIZE, hidden_dim=512, n_layers=8, dropout=DROPOUT)
m = m.to(device)
optimizer = torch.optim.Adam(m.parameters(), lr=LEARNING_RATE, weight_decay=L2_REG)  # weight_decay=1e-5 L2 regularization
m, optimizer,scheduler, epoch, loss, train_seed = load_checkpoint(m, optimizer, CHECKPOINT_PATH)
# Generate a sequence
print(f'Generating sequence of {FRAMES_GENERATE} frames...')

generated = m.generate(, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)



100%|██████████| 4102/4102 [00:03<00:00, 1086.69it/s]


Validating interpolation...
No errors found!


4102it [00:01, 2499.12it/s]
4102it [00:01, 2397.84it/s]


Creating keypoint frames...


100%|██████████| 4102/4102 [00:03<00:00, 1039.36it/s]


...

4102it [00:03, 1110.06it/s]


Validating length of data after delta
Adding emotions to frames...


100%|██████████| 4102/4102 [00:00<00:00, 9863.41it/s]


Validating length of data after emotions
frame_dim: 107
Loading checkpoint...
Checkpoint loaded from checkpoints/proto5_checkpoint.pth
Generating sequence of 300 frames...


NameError: name 'batch' is not defined

In [19]:
def encoding_to_emotions(encoding):
    """
    Convert an encoding to its corresponding emotion labels.
    
    Parameters:
    - encoding (list of int): The encoding of the emotions.
    
    Returns:
    - list of str: The labels of the emotions.
    """
    
    emotion_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprise']
    
    # Find the indices of non-zero values in the encoding and return the corresponding labels
    emotion_indices = [i for i, value in enumerate(encoding) if value > 0]
    return [emotion_labels[i] for i in emotion_indices]

# Test the function with an example encoding
test_encoding = [0.97, 0, 1, 1, 0, 0, 0]
encoding_to_emotions(test_encoding)


['Anger', 'Fear', 'Happiness']

In [12]:
frame[-7:]

[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]

In [33]:
# switch emotions of frames 
def switch_emotion(frame,old_emotion,new_emotion):
    """
    Switch the emotion of a frame.
    
    Parameters:
    - frame (list of int): The frame to switch the emotion of.
    - old_emotion (str): The emotion to switch from.
    - new_emotion (str): The emotion to switch to.
    
    Returns:
    - list of int: The frame with the emotion switched.
    """
    
    # Get the indices of the old and new emotion
    emotion_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprise']
    old_emotion_index = emotion_labels.index(old_emotion)
    new_emotion_index = emotion_labels.index(new_emotion)
    
    # Switch the emotion
    frame[-7 + old_emotion_index] = 0
    frame[-7 + new_emotion_index] = 1
    
    return frame

new_batch = []
for frame in batch:
    emotion = encoding_to_emotions(frame[-7:])
    print(emotion)
    new_batch.append(frame[:-7] + emotion_to_encoding('Disgust'))

['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Anger', 'Disgust', 'Happiness', 'Neutral']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Happiness', 'Neutral', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Sad', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Fear', 'Happiness']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Sur

In [24]:
for batch in unnorm_out:
    visualise_skeleton(batch, max_x, max_y, max_frames=FRAMES_GENERATE,save =True,save_path=None,prefix=f'adam_{EPOCHS}_coord_normalemotion_happy',train_seed=train_seed,delta=False)

Visualizing frames: 100%|██████████| 300/300 [00:33<00:00,  9.07it/s]


In [25]:
visualise_skeleton(new_batch, max_x, max_y, max_frames=FRAMES_GENERATE,save = True,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytosad',train_seed=train_seed,delta=False)

Visualizing frames:  27%|██▋       | 82/300 [00:09<00:24,  9.06it/s]


KeyboardInterrupt: 

In [34]:
happy_to_sad_xb = [new_batch[-16:]]
happy_to_sad_xb = torch.tensor(happy_to_sad_xb).to(device).float()


generated = m.generate(happy_to_sad_xb, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)


In [35]:
for new_batch in unnorm_out:

    visualise_skeleton(new_batch, max_x, max_y, max_frames=FRAMES_GENERATE,save = True,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust',train_seed=train_seed,delta=False)

Visualizing frames: 100%|██████████| 300/300 [00:32<00:00,  9.17it/s]


In [36]:
xb.shape

torch.Size([1, 16, 107])